# Import

In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import v2
import torch.nn as nn
from sklearn.model_selection import KFold, train_test_split

import segmentation_models_pytorch as smp
import wandb

In [2]:
import sys
sys.path.insert(0, "c:/Users/laish/1_Codes/Image_processing_toolchain")

from API_functions.DL import load_data, log, seed
from API_functions import file_batch as fb

# Hyperparameter and log

In [3]:
my_parameters = {
    'seed': 402,

    'Kfold': None,
    'ratio': 0.2,

    'model': '???',
    'optimizer': 'adam',
    'learning_rate':  0.001,
    'batch_size': 32,
    'loss_function': 'cross_entropy',

    'n_epochs': 1000,
    'patience': 50,
}

device = 'cuda'
mylogger = log.Logger('all')

seed.stablize_seed(my_parameters['seed'])

In [4]:
wandb.init(
    project="U-Net",
    name='5.what???',
    config=my_parameters,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: laishixuan123 (laishixuan123-china-agricultural-university). Use `wandb login --relogin` to force relogin


# Transform

In [5]:
# Define transformations and dataset
transform_common = v2.Compose([
    v2.ToTensor(),
    v2.ToDtype(torch.float32),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    v2.RandomRotation(degrees=90),
])

transform_train_special = v2.Compose([
    v2.ColorJitter(brightness=0.1, contrast=0.1)
])

transform_train = v2.Compose([
    transform_common,
    transform_train_special
])

transform_train_label = transform_common

transform_val = v2.Compose([
    v2.ToTensor(),
    v2.ToDtype(torch.float32)
])

transform_test = transform_val

c:\Users\laish\miniconda3\envs\dpln39\lib\site-packages\torchvision\transforms\v2\_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


# Load_data

In [6]:
def load_images(paths):
    return [cv2.imread(p, cv2.IMREAD_GRAYSCALE) for p in paths]

data_paths = fb.get_image_names('g:/DL_Data_raw/version0/train_images/', None, 'png')
labels_paths = fb.get_image_names('g:/DL_Data_raw/version0/train_labels/', None, 'png')
test_paths = fb.get_image_names('g:/DL_Data_raw/version0/test_images/', None, 'png')
test_labels_paths = fb.get_image_names('g:/DL_Data_raw/version0/test_labels/', None, 'png')

data = load_images(data_paths)
labels = load_images(labels_paths)
tests = load_images(test_paths)
test_labels = load_images(test_labels_paths)

train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=my_parameters['ratio'], random_state=my_parameters['seed'])

train_dataset = load_data.my_Dataset(train_data, train_labels, transform=transform_train, label_transform=transform_train_label)
val_dataset = load_data.my_Dataset(val_data, val_labels, transform=transform_val, label_transform=transform_val)
test_dataset = load_data.my_Dataset(tests, test_labels, transform=transform_test, label_transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=my_parameters['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=my_parameters['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=my_parameters['batch_size'], shuffle=False)

print(f'len of train_data: {len(train_data)}, len of val_data: {len(val_data)}, len of test_data: {len(tests)}')

10 images have been found in g:/DL_Data_raw/version0/train_images/
The first 3 images are:
g:/DL_Data_raw/version0/train_images\002_ou_DongYing_12633.png
g:/DL_Data_raw/version0/train_images\002_ou_DongYing_12634.png
g:/DL_Data_raw/version0/train_images\002_ou_DongYing_12635.png
Get names completely!
10 images have been found in g:/DL_Data_raw/version0/train_labels/
The first 3 images are:
g:/DL_Data_raw/version0/train_labels\002_ou_DongYing_12633.png
g:/DL_Data_raw/version0/train_labels\002_ou_DongYing_12634.png
g:/DL_Data_raw/version0/train_labels\002_ou_DongYing_12635.png
Get names completely!
5 images have been found in g:/DL_Data_raw/version0/test_images/
The first 3 images are:
g:/DL_Data_raw/version0/test_images\002_ou_DongYing_13635_roi_selected.png
g:/DL_Data_raw/version0/test_images\002_ou_DongYing_13636_roi_selected.png
g:/DL_Data_raw/version0/test_images\002_ou_DongYing_13637_roi_selected.png
Get names completely!
5 images have been found in g:/DL_Data_raw/version0/test_lab

# Model

In [7]:
# class DoubleConv(nn.Module):
#     """(convolution => [BN] => ReLU) * 2"""

#     def __init__(self, in_channels, out_channels):
#         super().__init__()
#         self.double_conv = nn.Sequential(
#             nn.Conv2d(in_channels, out_channels, 3, padding=1),
#             nn.BatchNorm2d(out_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(out_channels, out_channels, 3, padding=1),
#             nn.BatchNorm2d(out_channels),
#             nn.ReLU(inplace=True)
#         )

#     def forward(self, x):
#         return self.double_conv(x)

# class UNet(nn.Module):
#     def __init__(self):
#         super(UNet, self).__init__()
#         self.dconv_down1 = DoubleConv(1, 64)
#         self.dconv_down2 = DoubleConv(64, 128)
#         self.dconv_down3 = DoubleConv(128, 256)
#         self.dconv_down4 = DoubleConv(256, 512)        

#         self.maxpool = nn.MaxPool2d(2)
#         self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        
#         self.dconv_up3 = DoubleConv(256 + 512, 256)
#         self.dconv_up2 = DoubleConv(128 + 256, 128)
#         self.dconv_up1 = DoubleConv(64 + 128, 64)
        
#         self.conv_last = nn.Conv2d(64, 1, 1)
        
#     def forward(self, x):
#         conv1 = self.dconv_down1(x)
#         x = self.maxpool(conv1)
        
#         conv2 = self.dconv_down2(x)
#         x = self.maxpool(conv2)
        
#         conv3 = self.dconv_down3(x)
#         x = self.maxpool(conv3)
        
#         x = self.dconv_down4(x)
        
#         x = self.upsample(x)        
#         x = torch.cat([x, conv3], dim=1)
        
#         x = self.dconv_up3(x)
#         x = self.upsample(x)
#         x = torch.cat([x, conv2], dim=1)

#         x = self.dconv_up2(x)
#         x = self.upsample(x)
#         x = torch.cat([x, conv1], dim=1)

#         x = self.dconv_up1(x)
        
#         x = self.conv_last(x)

#         x = torch.sigmoid(x)  # Apply sigmoid activation to the output
#         return x

In [8]:
class DiceBCELoss(nn.Module):
    def __init__(self):
        super(DiceBCELoss, self).__init__()
        self.bce = nn.BCELoss()

    def forward(self, inputs, targets, smooth=1):
        # First, calculate the BCE loss
        inputs = torch.sigmoid(inputs)
        bce_loss = self.bce(inputs, targets)
        
        # Calculate Dice Loss
        inputs_flat = inputs.view(-1)
        targets_flat = targets.view(-1)
        
        intersection = (inputs_flat * targets_flat).sum()
        dice_loss = 1 - (2. * intersection + smooth) / (inputs_flat.sum() + targets_flat.sum() + smooth)
        
        # Combine BCE + Dice
        return 0.5 * bce_loss + 0.5 * dice_loss

# Train

In [9]:
model = smp.Unet(
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to C:\Users\laish/.cache\torch\hub\checkpoints\efficientnet-b4-6ed6700e.pth
100%|██████████| 74.4M/74.4M [03:01<00:00, 430kB/s]  


In [10]:
# model = UNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=my_parameters['learning_rate'])
criterion = DiceBCELoss()

In [ ]:
val_loss_best = 100000

for epoch in range(my_parameters['n_epochs']):
    model.train()
    train_loss = 0.0
    
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
    
    train_loss_mean = train_loss / len(train_loader.dataset)


    model.eval()
    val_loss = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)

    val_loss_mean = val_loss / len(val_loader.dataset)
    dict = {'train_loss': train_loss_mean, 'epoch': epoch, 'val_loss': val_loss_mean}
    mylogger.log(dict)

    if val_loss_mean < val_loss_best:
        val_loss_best = val_loss_mean
        torch.save(model.state_dict(), 'model.pth')
        print(f'Model saved at epoch {epoch}, val_loss: {val_loss_mean}')

100%|██████████| 1/1 [00:35<00:00, 35.57s/it]


{'epoch': 0, 'train_loss': 0.9476802349090576, 'val_loss': 1.3187569379806519}
Model saved at epoch 0, val_loss: 1.3187569379806519


100%|██████████| 1/1 [00:37<00:00, 37.14s/it]


{'epoch': 1, 'train_loss': 0.9074574708938599, 'val_loss': 1.3432536125183105}


100%|██████████| 1/1 [00:39<00:00, 39.66s/it]


...

100%|██████████| 1/1 [00:29<00:00, 29.47s/it]


{'epoch': 93, 'train_loss': 0.5230998396873474, 'val_loss': 0.5225046277046204}
Model saved at epoch 93, val_loss: 0.5225046277046204


100%|██████████| 1/1 [00:29<00:00, 29.86s/it]


{'epoch': 94, 'train_loss': 0.5228769183158875, 'val_loss': 0.5222724676132202}
Model saved at epoch 94, val_loss: 0.5222724676132202


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
wandb.finish()

# Test

In [ ]:
def save_image(image, path):
    """Save a tensor as an image."""
    image = image.squeeze().cpu().numpy()
    plt.imsave(path, image, cmap='gray')

def test_model(model, test_loader, device='cuda'):
    # model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Turn off gradients to speed up this part
        for i, (images, labels) in enumerate(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            outputs = torch.sigmoid(outputs)  # Apply sigmoid to get values between 0 and 1
            outputs = outputs > 0.5  # Threshold the probabilities to create a binary mask
            
            # Save output images
            for idx, output in enumerate(outputs):
                save_path = f'g:/DL_Data_raw/version0/inference/tests_inference5/002_ou_DongYing_{i*test_loader.batch_size + idx + 13635}_roi_selected.png'
                save_image(output, save_path)

            print(f'Processed batch {i+1}/{len(test_loader)}')


In [ ]:
# Test the model
model = smp.Unet(
    encoder_name="efficientnet-b2",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model.load_state_dict(torch.load('model.pth'))
model = model.to(device)
test_model(model, test_loader)

In [ ]:
from API_functions import file_compare as fc
%matplotlib qt

db = fc.ImageDatabase()
# image_processor.add_result('pre_processed', tpi.user_threshold(image_processor.image, 160))
zoom = fc.ZoomRegion(350, 450, 100, 200)
db.add_additional_folder('f:/3.Experimental_Data/DL_Data_raw/tests/', 'test_set')
db.add_additional_folder('f:/3.Experimental_Data/DL_Data_raw/tests_inference4/', 'test_inference')
db.add_additional_folder('f:/3.Experimental_Data/DL_Data_raw/test_labels/', 'test_labels')
image_processor = db.get_image_processor('002_ou_DongYing_13636_roi_selected.png')
image_processor.show_images('test_set', 'test_inference', 'test_labels', zoom_region=zoom)